<a href="https://colab.research.google.com/github/DotaDeGuapa/dw_matrix/blob/master/day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [70]:
!pip install eli5

In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook

In [72]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix"

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [73]:
ls data

men_shoes.csv


In [74]:
df = pd.read_csv('data/men_shoes.csv', low_memory=False)
df.shape

(18280, 48)

In [75]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension', 'ean', 'features',
       'flavors', 'imageurls', 'isbn', 'keys', 'manufacturer',
       'manufacturernumber', 'merchants', 'name', 'prices_amountmin',
       'prices_amountmax', 'prices_availability', 'prices_color',
       'prices_condition', 'prices_count', 'prices_currency',
       'prices_dateadded', 'prices_dateseen', 'prices_flavor', 'prices_issale',
       'prices_merchant', 'prices_offer', 'prices_returnpolicy',
       'prices_shipping', 'prices_size', 'prices_source', 'prices_sourceurls',
       'prices_warranty', 'quantities', 'reviews', 'sizes', 'skus',
       'sourceurls', 'upc', 'vin', 'websiteids', 'weight'],
      dtype='object')

In [0]:
def run_model(feats, model = DecisionTreeRegressor(max_depth=5)):
  X = df [feats].values
  y = df ['prices_amountmin'].values

  model = DecisionTreeRegressor(max_depth=5)

  scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error')

  return np.mean(scores), np.std(scores)

In [77]:
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]
run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [78]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'], model)

(-58.133398968282776, 4.206122611474276)

In [79]:
df.head()

,id,asins,brand,categories,colors,count,dateadded,dateupdated,descriptions,dimension,ean,features,flavors,imageurls,isbn,keys,manufacturer,manufacturernumber,merchants,name,prices_amountmin,prices_amountmax,prices_availability,prices_color,prices_condition,prices_count,prices_currency,prices_dateadded,prices_dateseen,prices_flavor,prices_issale,prices_merchant,prices_offer,prices_returnpolicy,prices_shipping,prices_size,prices_source,prices_sourceurls,prices_warranty,quantities,reviews,sizes,skus,sourceurls,upc,vin,websiteids,weight,brand_cat
0,AVpfHrJ6ilAPnD_xVXOI,NaN,Josmo,"Clothing,Shoes,Men's Shoes,All Men's Shoes",NaN,NaN,2016-11-07T00:45:12Z,2016-11-07T00:45:12Z,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""sourceU...",NaN,6.993020e+11,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Shoe...",NaN,https://i5.walmartimages.com/asr/13ac3d61-003c...,NaN,"josmo/8190wnavy75,699302044036,0699302044036",NaN,8190-W-NAVY-7.5,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""name"":""...","Josmo 8190 Plain Infant Walking Shoes, Navy - ...",39.89,39.89,NaN,NaN,NaN,NaN,USD,2016-11-07T00:45:12Z,2016-11-05T00:00:00Z,NaN,True,NaN,REDUCED USD 12.10,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,NaN,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,6.993020e+11,NaN,NaN,NaN,0
1,AVpfHrJ6ilAPnD_xVXOI,NaN,Josmo,"Clothing,Shoes,Men's Shoes,All Men's Shoes",NaN,NaN,2016-11-07T00:45:12Z,2016-11-07T00:45:12Z,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""sourceU...",NaN,6.993020e+11,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Shoe...",NaN,https://i5.walmartimages.com/asr/13ac3d61-003c...,NaN,"josmo/8190wnavy75,699302044036,0699302044036",NaN,8190-W-NAVY-7.5,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""name"":""...","Josmo 8190 Plain Infant Walking Shoes, Navy - ...",51.99,51.99,NaN,NaN,new,NaN,USD,2016-11-07T00:45:12Z,2016-11-05T00:00:00Z,NaN,False,UnbeatableSale - Walmart.com,REDUCED USD 12.10,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,NaN,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,6.993020e+11,NaN,NaN,NaN,0
2,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""2016-06-14T04:29:57Z""],""name"":""...",Servus By Honeywell Shoe Studs Zsr101blmlg,40.02,40.02,NaN,NaN,new,NaN,USD,2016-06-14T04:29:57Z,2016-03-08T00:00:00Z,NaN,False,SIM Supply Inc - Walmart.com,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,1
3,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""2016-06-14T04:29:57Z""],""name"":""...",Servus By Honeywell Shoe Studs Zsr101blmlg,50.31,50.31,NaN,NaN,new,NaN,USD,2016-06-14T04:29:57Z,2015-11-30T00:00:00Z,NaN,False,SIM Supply Inc - Walmart.com,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,1
4,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""

In [80]:
df.features.head().values


array(['[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SERVUS BY HONEYWELL"]},{"key":"manufacturer_part_number","value":["ZSR101BLMLG"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SER

In [81]:
str_dict = '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]'

literal_eval(str_dict)

[{'key': 'Gender', 'value': ['Men']},
 {'key': 'Shoe Size', 'value': ['M']},
 {'key': 'Shoe Category', 'value': ["Men's Shoes"]},
 {'key': 'Color', 'value': ['Multicolor']},
 {'key': 'Manufacturer Part Number', 'value': ['8190-W-NAVY-7.5']},
 {'key': 'Brand', 'value': ['Josmo']}]

In [82]:
test = {'key' : 'value'}
test['key']
str(test)

"{'key': 'value'}"

In [0]:
def parse_features(x):
  output_dict= {}
  if str(x) == 'nan' : return output_dict

  features = literal_eval(x.replace('\\"', '"'))
  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()

    output_dict[key] = value

  return output_dict

df['features_parsed'] = df['features'].map(parse_features)

In [85]:
keys = set()


df['features_parsed'].map(lambda x: keys.update(x.keys()))

len(keys)

476

In [87]:
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook (keys):
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key]if key in feats else np.nan)


In [88]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_product in inches (l x w x h)', 'feat_100% adidas authentic',
       'feat_productgroup', 'feat_international shipping?',
       'feat_batteries required', 'feat_year made',
       'feat_guaranteed authentic', 'feat_sub style', 'feat_wheeled',
       'feat_label'],
      dtype='object', length=526)

In [0]:
keys_stat = {}
for key in keys:
  keys_stat[key] = df[ False == df[get_name_feat(key)].isnull() ].shape[0] /df.shape[0] *100


In [101]:
{k: v for k,v in keys_stat.items() if v > 30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [91]:
df.shape[0]

18280

In [141]:
df.features_parsed.head().values

array([{'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'}],
      dtype=object)

In [0]:
df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].factorize()[0]
df['feat_manufacturer part number_cat'] = df['feat_manufacturer part number'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]
df['feat_sport_cat'] = df['feat_sport'].factorize()[0]
df['feat_style_cat'] = df['feat_style'].factorize()[0]

for key in keys:
  df[get_name_feat(key) + '_cat'] = df[get_name_feat(key)].factorize()[0]
  get_name_feat(key)




In [143]:
df['brand'] = df['brand'].map(lambda x: str(x).lower())

df[ df.brand != df.feat_brand].shape

(9434, 533)

In [144]:
model = RandomForestRegressor(max_depth=5, n_estimators=100)

run_model(['brand_cat'], model)

(-58.133398968282776, 4.206122611474276)

In [155]:
feats_cat = [x for x in df.columns if 'cat' in x]
feats_cat

['categories',
 'brand_cat',
 'feat_location - country',
 'feat_fabrication',
 'feat_shoe category',
 'feat_catalog',
 'feat_certifications and listings',
 'feat_clothing category',
 'feat_multi pack indicator',
 'feat_recommended location',
 'feat_location - city/state',
 'feat_brand_cat',
 'feat_color_cat',
 'feat_gender_cat',
 'feat_manufacturer part number_cat',
 'feat_material_cat',
 'feat_sport_cat',
 'feat_style_cat',
 'feat_date first available at amazon.co.uk_cat',
 'feat_number of pieces_cat',
 'feat_fastener_cat',
 'feat_diamond clarity_cat',
 'feat_foot arch_cat',
 'feat_mpn_cat',
 'feat_colour_cat',
 'feat_insulation_cat',
 'feat_metal stamp_cat',
 'feat_target audience_cat',
 'feat_model number_cat',
 'feat_navy blue_cat',
 'feat_item type_cat',
 'feat_free item_cat',
 'feat_genre_cat',
 'feat_very popular bag now a days_cat',
 'feat_design_cat',
 'feat_interior pockets_cat',
 'feat_inseam_cat',
 'feat_eye size:_cat',
 'feat_is orthopedic_cat',
 'feat_upper material_cat',

In [160]:
feats = ['brand_cat', 'feat_brand_cat', 'feat_gender_cat', 'feat_material_cat' , 'feat_sport_cat']

feats +=['feat_used with equipment type_cat',
 'feat_manufacturer number_cat',
 'feat_hood_cat',
 'feat_style:_cat',
 'feat_lens type_cat',
 'feat_overall material_cat',
 'feat_work shoes_cat',
 'feat_weather resistant_cat',
 'feat_style number_cat',
 'feat_shape_cat',
 'feat_item character_cat',
 'feat_item spec attr - style (ebay)_cat',
 'feat_number of pockets_cat',
 'feat_ground_cat',
 'feat_cleaning, care & maintenance_cat',
 'feat_alarm_cat',
 'feat_finish_cat',
 'feat_military style_cat',
 'feat_retail price_cat',
 'feat_width_cat',
 'feat_construction_cat',
 'feat_profession_cat',
 'feat_shipping weight (in pounds)_cat',
 'feat_protects against_cat',
 'feat_occasion_cat',
 'feat_has adaptive lenses_cat',
 'feat_frame material:_cat',
 'feat_leather grade_cat',
 'feat_article_cat',
 'feat_year_cat',
 'feat_crystal_cat',
 'feat_shoe size_cat',
 'feat_full product manual_cat',
 'feat_date_cat',
 'feat_boxed-product dimensions_cat',
 'feat_net weight_cat',
 'feat_pronation_cat',
 'feat_is energy star-certified_cat',
 'feat_length_cat',
 'feat_lens_cat',
 'feat_atpv arc rating_cat',
 'feat_removable hood_cat',
 'feat_expandable_cat',
 'feat_clasp type_cat',
 'feat_clothing size type_cat',
 'feat_cushioning_cat',
 'feat_name_cat',
 'feat_frame type_cat',
 'feat_fastener type_cat',
 'feat_resolution_cat',
 'feat_nike id_cat',
 'feat_suitable for_cat',
 'feat_ul safety listing_cat',
 'feat_rx-able_cat',
 'feat_country////region of manufacture_cat',
 'feat_safety feature_cat',
 'feat_era_cat',
 'feat_domestic shipping_cat',
 'feat_antiscratch lens coating_cat',
 'feat_disclosures_cat']

#feats += feats_cat
#feats = list(set(feats))

model = RandomForestRegressor(max_depth=5, n_estimators=100)

run_model(['brand_cat', 'feat_brand_cat', 'feat_color_cat', 'feat_gender_cat', 'feat_material_cat', 'feat_sport_cat' ], model)

(-58.185605800254265, 4.229172928821459)

In [161]:
X = df[ feats ].values
y = df['prices_amountmin'].values


m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X,y)

perm = PermutationImportance (m, random_state=1).fit(X,y);

eli5.show_weights(perm, feature_names=feats)

Weight,Feature
0.2567 ± 0.0066,brand_cat
0.1036 ± 0.0124,feat_material_cat
0.0219 ± 0.0034,feat_gender_cat
0.0190 ± 0.0009,feat_brand_cat
0.0074 ± 0.0004,feat_shape_cat
0.0039 ± 0.0006,feat_fastener type_cat
0.0018 ± 0.0002,feat_lens_cat
0.0014 ± 0.0002,feat_crystal_cat
0.0013 ± 0.0002,feat_shoe size_cat
0.0012 ± 0.0001,feat_occasion_cat


In [138]:
df['brand'].value_counts(normalize=True)

nike                    0.097210
puma                    0.033315
ralph lauren            0.028775
vans                    0.021116
new balance             0.020295
                          ...   
adidas by raf simons    0.000055
candyman                0.000055
baigio                  0.000055
inew(not oakley)        0.000055
tempco                  0.000055
Name: brand, Length: 1732, dtype: float64

In [151]:
df[df['brand'] == 'nike'].features_parsed.sample(5).values

array([{'style': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'style': 'soccer shoes', 'country/region of manufacture': 'italy', 'type': 'cleats', 'condition': 'new with box', 'main color': 'orange', 'material': 'synthetic'},
       {'sport': 'tennis', 'style': 'jacket', 'condition': 'new with tags'},
       {'size': '10', 'model number': '705149 010', 'shoe width': 'medium', 'material': 'mesh', 'sport': 'running shoes', 'occasion': 'running shoes', 'gender': 'men', 'shoe size': '8.5 us men', 'shoe category': "men's shoes", 'fabric content': 'mesh', 'color': 'black/anthracite/black', 'model': '705149 010', 'shoe closure': 'lace-up', 'casual & dress shoe style': 'running shoes', 'manufacturer part number': '705149 010', 'brand': 'nike', 'age group': 'men'},
       {'season': 'all-season', 'material': 'synthetic', 'gender': 'men', 'shoe size': '8.5', 'color': 'blue', 'manufacturer part number': '554954 400', 'brand': 'nike', 'age group': 'adult'}],
      dtype=ob

In [153]:
df['feat_age group'].value_counts()

adult               4563
men                  350
child                 77
men's                 33
unisex                 6
toddler                4
infant                 4
mens                   4
boys'                  3
women ,�� unisex       2
men||women             2
youth                  2
women                  2
12 up                  1
adult ,�� teen         1
Name: feat_age group, dtype: int64